In [22]:
import pandas as pd

df = pd.read_csv("../Data_120294_2023-03-29_cleaned.csv")


In [23]:
mean_wl_time = df['wl_time'].mean()
print(f"Mean waiting list time: {mean_wl_time}")

df['wl_time'] = df['wl_time'].apply(lambda x: 0 if x < mean_wl_time else 1)
print(df['wl_time'].value_counts(normalize=True))

Mean waiting list time: 192.52839426430694
0    0.716807
1    0.283193
Name: wl_time, dtype: float64


In [24]:
import keras.backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_score(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [25]:
from sklearn.model_selection import train_test_split
from autokeras import StructuredDataClassifier
from sklearn.metrics import f1_score

X = df.drop(columns=['wl_time'])
y = df['wl_time']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = StructuredDataClassifier(max_trials=25,
                               metrics=["accuracy"],
                               overwrite=False)

clf.fit(X_train, y_train, verbose=1)
print(clf.evaluate(X_test, y_test))
f1_s = f1_score(y_test, clf.predict(X_test))
print(f"F1 score: {f1_s}")
print("F1 Macro:", f1_score(y_test, clf.predict(X_test), average='macro'))


Trial 25 Complete [00h 01m 28s]
val_accuracy: 0.7661003470420837

Best val_accuracy So Far: 0.7669093608856201
Total elapsed time: 00h 52m 32s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Epoch 1/11
1936/1936 [==============================] - 3s 1ms/step - loss: 0.5153 - accuracy: 0.7455
Epoch 2/11
1936/1936 [==============================] - 2s 1ms/step - loss: 0.4919 - accuracy: 0.7583
Epoch 3/11
1936/1936 [==============================] - 2s 1ms/step - loss: 0.4834 - accuracy: 0.7619
Epoch 4/11
1936/1936 [==============================] - 2s 1ms/step - loss: 0.4787 - accuracy: 0.7647
Epoch 5/11
1936/1936 [==============================] - 2s 1ms/step - loss: 0.4753 - accuracy: 0.7666
Epoch 6/11
1936/1936 [==============================] - 2s 1ms/step - loss: 0.4729 - accuracy: 0.7684
Epoch 7/11
1936/1936 [==============================] - 2s 1ms/step - loss: 0.4708 - accuracy: 0.7706
Epoch 8/11
1936/1936 [==============================] - 2s 1ms/step - loss: 0.4688 - accuracy: 0.7721
Epoch 9/11
1936/1936 [==============================] - 2s 1ms/step - loss: 0.4672 - accuracy: 0.7733
Epoch 10/11
1936/1936 [==============================] - 2s 1ms/step - loss: 0.465

INFO:tensorflow:Assets written to: ./structured_data_classifier/best_model/assets


INFO:tensorflow:Assets written to: ./structured_data_classifier/best_model/assets


484/484 [==============================] - 1s 1ms/step - loss: 0.4756 - accuracy: 0.7669
[0.47559458017349243, 0.7668905854225159]
484/484 [==============================] - 1s 1ms/step
F1 score: 0.516802784844022
484/484 [==============================] - 1s 1ms/step
F1 Macro: 0.6815978171932389


In [26]:
# save the model
clf.export_model("autokeras_model.h5")

f1_s = f1_score(y_test, clf.predict(X_test))
print(f"F1 score: {f1_s}")
print("F1 Macro:", f1_score(y_test, clf.predict(X_test), average='macro'))

484/484 [==============================] - 1s 1ms/step
F1 score: 0.516802784844022
484/484 [==============================] - 1s 1ms/step
F1 Macro: 0.6815978171932389
